In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
df=df[2885849:]

test_data=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

test = test_data.drop("ID", axis=1)
df.head()


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing 
import regex as re



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

In [ ]:
df=df.drop(columns=['date_block_num'], axis=1)
df['item_id'].fillna((df['item_id'].mean()), inplace=True)
df['item_id'].fillna((df['item_id'].mean()), inplace=True) 
df['item_price'].fillna((df['item_price'].mean()), inplace=True) 
df['item_cnt_day'].fillna((df['item_cnt_day'].mean()), inplace=True)

In [ ]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
add_datepart(df,"date")
df=df.drop("Elapsed", axis=1)


In [ ]:
df.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
df['Is_month_end']= label_encoder.fit_transform(df['Is_month_end']) 
df['Is_month_start']= label_encoder.fit_transform(df['Is_month_start']) 
df['Is_quarter_end']= label_encoder.fit_transform(df['Is_quarter_end']) 
df['Is_quarter_start']= label_encoder.fit_transform(df['Is_quarter_start']) 
df['Is_year_end']= label_encoder.fit_transform(df['Is_year_end']) 
df['Is_year_start']= label_encoder.fit_transform(df['Is_year_start'])

In [ ]:
df.head()

In [ ]:
test['item_price']=df['item_price']
test['item_price'].fillna((df['item_price'].mean()), inplace=True)
test['Year']=df['Year']
test['Year'].fillna((df['Year'].mean()), inplace=True)
test['Month']=df['Month']
test['Month'].fillna((df['Month'].mean()), inplace=True)
test['Week']=df['Week']
test['Week'].fillna((df['Week'].mean()), inplace=True)
test['Day']=df['Day']
test['Day'].fillna((df['Day'].mean()), inplace=True)
test['Dayofweek']=df['Dayofweek']
test['Dayofweek'].fillna((df['Dayofweek'].mean()), inplace=True)
test['Dayofyear']=df['Dayofyear']
test['Dayofyear'].fillna((df['Dayofyear'].mean()), inplace=True)
test['Is_month_end']=df['Is_month_end']
test['Is_month_end'].fillna((df['Is_month_end'].mean()), inplace=True)
test['Is_month_start']=df['Is_month_start']
test['Is_month_start'].fillna((df['Is_month_start'].mean()), inplace=True)
test['Is_quarter_end']=df['Is_quarter_end']
test['Is_quarter_end'].fillna((df['Is_quarter_end'].mean()), inplace=True)
test['Is_quarter_start']=df['Is_quarter_start']
test['Is_quarter_start'].fillna((df['Is_quarter_start'].mean()), inplace=True)
test['Is_year_end']=df['Is_year_end']
test['Is_year_end'].fillna((df['Is_year_end'].mean()), inplace=True)
test['Is_year_start']=df['Is_year_start']
test['Is_year_start'].fillna((df['Is_year_start'].mean()), inplace=True)
test.head()

In [ ]:
item_cnt=df['item_cnt_day']
df=df.drop(labels='item_cnt_day',axis=1)
df.insert(15,'item_cnt_day',item_cnt);

In [ ]:
(80*len(df))/100

In [ ]:
train_df=df.loc[2885849:2925849]
valid_df=df.loc[2925849:]
len(train_df),len(valid_df)

In [ ]:
y_train=train_df['item_cnt_day']
x_train=train_df.drop('item_cnt_day',axis=1)
y_valid=valid_df['item_cnt_day']
x_valid=valid_df.drop('item_cnt_day',axis=1)

In [ ]:
model1=RandomForestClassifier()
model2=XGBClassifier()
model3=neighbors.KNeighborsClassifier()
model4=SVC()

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_valid.shape

In [ ]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)
model4.fit(x_train,y_train)

pred1=model1.predict(x_valid)
pred2=model2.predict(x_valid)
pred3=model3.predict(x_valid)
pred4=model4.predict(x_valid)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(pred1,y_valid))
print(accuracy_score(pred2,y_valid))
print(accuracy_score(pred3,y_valid))
print(accuracy_score(pred4,y_valid))

In [ ]:
predictions = model4.predict(test)

submission = pd.DataFrame({
        "ID": test_data.ID,
        "item_cnt_month": predictions
    })

submission.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
submission = pd.read_csv('submission.csv')
submission.head()
